In [1]:
import numpy as np
import cv2
import tensorflow as tf
import time

from test import *

In [2]:
# Static values
PATH = 'testVideos/IMG_9367.MOV'

#how many frame to play until pause
SHOW_FRAME = 60


In [3]:
def predict_image(img):
    pass

In [4]:
def captionFrame(frame):
    img = tf.keras.layers.Resizing(224, 224)(frame)
    img = tf.keras.applications.resnet50.preprocess_input(img)
    predict_image(img)

## Run the app

In [5]:
#loop over all frames  'space' = next frame | 'q' = quit
cap = cv2.VideoCapture(PATH)
if (cap.isOpened()== False):
    print("Error opening video stream or file")
else:
    print("Statred capturing")

total_frames = 0
while cap.isOpened():
    HasFrames, frame = cap.read()
    #if vidoe is not done do
    if HasFrames:
        total_frames += 1
        cv2.imshow('Video', frame)
        #when you reach the pause frame do
        if((total_frames % SHOW_FRAME) == 0):
            #Caption 'frame'
            start = time.time()
            captionFrame(frame)
            print(f'Time taken for 1 image {time.time()-start:.4f} sec\n')
            #press 'E' to get next frame
            if(cv2.waitKey(5000) == ord('e')):
                continue

        if(cv2.waitKey(25) == ord('q')):
            break    

    else:
        break

cap.release()
cv2.destroyAllWindows()

Statred capturing
Time taken for 1 image 10.1470 sec



Error: Canceled future for execute_request message before replies were done